<a href="https://colab.research.google.com/github/TOT28-Sys/SpyMAX-Source-Code/blob/main/Copie_de_islamicGpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets wandb  # Install required libraries
import os
import pandas as pd
from transformers import AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import Dataset
import torch
import re
import wandb  # Import W&B library

# Authenticate W&B
wandb.login(key="e097b3e9b4c8fb921e47b03da64c31f880f7de72")

# Function to clean text
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'[\n\t]', ' ', text)  # Remove newlines and tabs
    text = text.strip()
    return text

# Load and clean data
def load_csv_files(folder_path):
    all_data = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            data = pd.read_csv(file_path, encoding='utf-8')
            all_data.append(data)
    if not all_data:
        raise ValueError(f"No CSV files found in directory: {folder_path}")
    return pd.concat(all_data, ignore_index=True)

csv_folder_path = r'/content/sample_data'
data = load_csv_files(csv_folder_path)

# Ensure 'text' column exists and is of type string
if 'text' not in data.columns:
    first_column = data.columns[0]
    data['text'] = data[first_column].astype(str)

data = data.dropna(subset=['text'])
data = data.drop_duplicates(subset=['text'])

texts = [clean_text(str(text)) for text in data['text'].tolist()]

# Create a Dataset from the texts
dataset = Dataset.from_dict({'text': texts})

# Define the tokenizer
model_name = 'aubmindlab/bert-base-arabertv02'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define the tokenization function with error handling
def tokenize_function(examples):
    try:
        return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128)
    except Exception as e:
        print(f"Error tokenizing: {examples['text']} - {e}")
        return tokenizer('empty', padding='max_length', truncation=True, max_length=128)

# Tokenize the dataset in batches
tokenized_dataset = dataset.map(tokenize_function, batched=True, batch_size=1000)

# Set the format for PyTorch
tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])

# Load the model
model = AutoModelForMaskedLM.from_pretrained(model_name)

# Define training arguments with W&B logging
training_args = TrainingArguments(
    output_dir='./results',
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=32,
    save_steps=10_000,
    save_total_limit=2,
    logging_dir='./logs',  # Directory for logs
    logging_steps=500,     # Log every 500 steps
    report_to="wandb",     # Enable W&B logging
    run_name="islamic_gpt_training",  # Custom run name for W&B
)

# Define data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,
    mlm_probability=0.15
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset,
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained('islamic_gpt_model')
tokenizer.save_pretrained('islamic_gpt_model')

# Finish W&B run
wandb.finish()

wandb: Currently logged in as: ahmed12 (ahmed12-645). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Map:   0%|          | 0/26363 [00:00<?, ? examples/s]

Some weights of the model checkpoint at aubmindlab/bert-base-arabertv02 were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Step,Training Loss
500,2.384100
1000,2.262800
1500,2.219800
2000,2.237300


train/epoch,▁▃▅▆█
train/global_step,▁▃▅▆█
train/grad_norm,█▃▃▁
train/learning_rate,█▆▃▁
train/loss,█▃▁▂
total_flos,5206184825389056.0
train/epoch,3
train/global_step,2472
train/grad_norm,4.20733
train/learning_rate,1e-05
train/loss,2.2373


In [18]:
# 1. تحميل المكتبات المطلوبة
import pandas as pd
import re
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset
import glob

# 2. تنظيف البيانات
def clean_text(text):
    """
    تنظيف النص من الرموز غير المرغوب فيها والفراغات الزائدة.
    """
    if not isinstance(text, str):  # إذا كانت القيمة غير نصية
        return ""  # إرجاع سلسلة فارغة
    text = re.sub(r'[\n\t]', ' ', text)  # إزالة الأسطر الجديدة والفراغات
    text = re.sub(r'\s+', ' ', text)     # إزالة الفراغات الزائدة
    return text.strip()

# 3. تحميل البيانات من ملفات CSV
def load_and_clean_data(csv_files):
    """
    تحميل البيانات من ملفات CSV وتنظيفها.
    """
    data_frames = []
    for file in csv_files:
        data = pd.read_csv(file, encoding='utf-8')
        data_frames.append(data)

    # دمج جميع البيانات في إطار واحد
    combined_data = pd.concat(data_frames, ignore_index=True)

    # إزالة الصفوف التي تحتوي على قيم فارغة في العمود '1'
    combined_data = combined_data.dropna(subset=['1'])

    # تحويل القيم إلى نصوص
    combined_data['1'] = combined_data['1'].astype(str)

    # تنظيف النصوص
    texts = combined_data['1'].tolist()  # استبدل '1' باسم العمود الصحيح
    cleaned_data = " ".join([clean_text(text) for text in texts])

    return cleaned_data

# 4. تحميل النموذج والـ Tokenizer
def load_model_and_tokenizer(model_name="gpt2"):
    """
    تحميل النموذج والـ Tokenizer.
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # إضافة Padding Token إذا لم يكن موجودًا
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    return tokenizer, model

# 5. توكنيز البيانات
def tokenize_data(texts, tokenizer, max_length=512):
    """
    توكنيز النصوص وتحويلها إلى تنسيق مناسب للتدريب.
    """
    inputs = tokenizer(texts, return_tensors='pt', max_length=max_length, truncation=True, padding=True)
    return inputs

# 6. إنشاء مجموعة بيانات
def create_dataset(inputs):
    """
    إنشاء مجموعة بيانات من المدخلات المميزة.
    """
    dataset = Dataset.from_dict({
        'input_ids': inputs['input_ids'].squeeze(),
        'attention_mask': inputs['attention_mask'].squeeze()
    })
    return dataset

# 7. تدريب النموذج
def train_model(model, tokenizer, dataset, output_dir="./islamic_gpt", num_train_epochs=3, batch_size=4):
    """
    تدريب النموذج على البيانات.
    """
    # إعداد DataCollator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,  # لأننا نستخدم GPT-2 (ليس BERT)
    )

    # إعداد TrainingArguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
        save_steps=10_000,
        save_total_limit=2,
        logging_dir="./logs",
        logging_steps=500,
        report_to="wandb",  # تفعيل W&B
        run_name="islamic_gpt_training",  # اسم التجربة
    )

    # إعداد Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        data_collator=data_collator,
    )

    # بدء التدريب
    trainer.train()

    # حفظ النموذج
    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

# 8. توليد النصوص
def generate_text(model, tokenizer, input_text, max_length=100):
    """
    توليد نصوص باستخدام النموذج المدرب.
    """
    inputs = tokenizer(input_text, return_tensors='pt')
    outputs = model.generate(**inputs, max_length=max_length, num_return_sequences=1)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text

# 9. التنفيذ الرئيسي
if __name__ == "__main__":
    # تحديد مسار ملفات CSV
    csv_files = glob.glob('/content/data/*.csv')  # تحديد جميع ملفات CSV في المجلد

    # تحميل البيانات وتنظيفها
    cleaned_data = load_and_clean_data(csv_files)

    # تقسيم النص إلى جمل
    sentences = re.split(r'[.!؟]', cleaned_data)
    sentences = [s.strip() for s in sentences if s.strip()]

    # تحميل النموذج والـ Tokenizer
    tokenizer, model = load_model_and_tokenizer(model_name="gpt2")

    # توكنيز البيانات
    inputs = tokenize_data(sentences, tokenizer)

    # إنشاء مجموعة بيانات
    dataset = create_dataset(inputs)

    # تدريب النموذج
    train_model(model, tokenizer, dataset)

    # توليد نص
    input_text = "ما هي أهمية الصدقة في الإسلام؟"
    generated_text = generate_text(model, tokenizer, input_text)
    print(f"النص المولد: {generated_text}")

Step,Training Loss


KeyboardInterrupt: 